In [1]:
# always check your python version
!python --version

Python 3.11.9


In [2]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# always check your boto3 version
%pip show boto3

In [3]:
!sudo apt-get install sqlite3 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libreadline8 libsqlite3-0 readline-common
Suggested packages:
  readline-doc sqlite3-doc
The following NEW packages will be installed:
  libreadline8 libsqlite3-0 readline-common sqlite3
0 upgraded, 4 newly installed, 0 to remove and 0 not upgraded.
Need to get 1617 kB of archives.
After this operation, 4067 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 readline-common all 8.1.2-1 [53.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libreadline8 amd64 8.1.2-1 [153 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libsqlite3-0 amd64 3.37.2-2ubuntu0.3 [641 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.3 [768 kB]
Fetched 1617 kB in 1s (1537 kB/s)
debconf: delaying package configuration, since apt-utils is n

In [4]:
!sqlite3 --version

3.37.2 2022-01-06 13:25:41 872ba256cbf61d9290b571c0e6d82a20c224ca3ad82971edc46b29818d5dalt1


In [5]:
from helper import load_text_file
import json
import boto3
client = boto3.client('bedrock-runtime')

In [ ]:
# Studio Workshop does not have permission for Command+R
# This is left here for example
#def converse(session_id, input):
#  history = Chat.read(session_id)  
#
#  system_prompt = load_text_file("prompts/jp-teacher.txt")
#  model_id = 'cohere.command-r-v1:0'
#  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nAssistant:"
#  resp = client.invoke_model(
#      modelId=model_id,
#      contentType='application/json',
#      accept='application/json',
#      body=json.dumps({
#        "message": prompt,
#        "max_tokens": 200,
#        "temperature": 1.0
#      })
#  )
#  body = json.loads(resp.get("body").read())
#  llm_response = body['text']
#
#  updated_history = f"{history}\n{llm_response}"
#  Chat.write(session_id,updated_history)
#
# return body

In [28]:
model_id = 'amazon.titan-text-express-v1'

# Differen models are optimized for differents tasks.
# Titan Express is optimized for english which is why it not replying in the expected way.
system_prompt = load_text_file("prompts/jp-teacher.txt")
history = """
User: I want to eat at Kyoto Katsugyu in Toronto
Bot: That sounds like a great idea. Kyoto Katsugyu is a Japanese restaurant located in Toronto, Canada.
User: I wan't asking a question, that was the english text I wanted to translate to Japanese.
Bot: I apologize for the confusion. To clarify, would you like me to translate the English text into Japanese?
User: Where are my japanese clues?
"""

prompt = f"{system_prompt}\n{history}\nBot:"

resp = client.invoke_model(
  modelId=model_id,
  contentType='application/json',
  accept='application/json',
  body=json.dumps({
    "inputText": prompt
  })
)
print("Usage ====== \n")
print("Input Tokens: "+ resp['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count'])
print("Output Tokens: " + resp['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count'])
print("\nResponse ====== \n")
body = json.loads(resp.get("body").read())
print(body['results'][0]['outputText'])

Usage ====== 

Input Tokens: 213
Output Tokens: 41

Response ====== 

 I apologize for the confusion. As a Japanese language teacher, my role is to assist you in learning Japanese. I will provide you with hints and clues to help you translate the English text into Japanese.


In [21]:
# Note: We will update our prompt document to tell is not to give us the direct answer.
messages = [
  {"role": "user", "content": [{"text": "I want to eat at Kyoto Katsugyu in Toronto"}]},
  {'role': 'assistant', 'content': [{'text': 'クルー:\n1) 私 (わたし) は トロントの 京都カツ牛 (きょうとかつぎゅう) で 食べたい (たべたい) と言っています。\n\n2) 京都カツ牛 は Toronto にある日本食レストランの名前です。\n\n3) 食べる (たべる) は「to eat」の意味の動詞の辞書形です。\n\nヒント:\n- 主語は私 (I)\n- トロントの〜で (in Toronto at〜) \n- レストラン名が出てきます\n- 動詞は「たい」形で欲求を表しています'}]},
  {"role": "user", "content": [{"text": "can you just tell me the translation?"}]}
]
system_prompt_text = load_text_file("prompts/jp-teacher.txt")
system_prompt = [{"text": system_prompt_text}]
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
resp = client.converse(
  modelId=model_id,
  messages=messages,
  system=system_prompt
)
print("Usage ====== \n")
print(resp['usage'])
print("\nResponse ====== \n")
text = resp['output']['message']['content'][0]['text']  
print(text)
print("\nMessage Object ====== \n")
msg = resp['output']['message']
print(msg)

Usage ====== 

{'inputTokens': 342, 'outputTokens': 89, 'totalTokens': 431}

Response ====== 

私(わたし)はトロントの京都肉牛(きょうとかつぎゅう)で食べたい(たべたい)です。

Watashi wa Toronto no Kyōto Katsugyu de tabetai desu.

Translation: I want to eat at Kyoto Katsugyu in Toronto.

Message Object ====== 

{'role': 'assistant', 'content': [{'text': '私(わたし)はトロントの京都肉牛(きょうとかつぎゅう)で食べたい(たべたい)です。\n\nWatashi wa Toronto no Kyōto Katsugyu de tabetai desu.\n\nTranslation: I want to eat at Kyoto Katsugyu in Toronto.'}]}


In [29]:
from chat_history import Chat
def converse(session_id, input):
  history = Chat.read(session_id)  

  system_prompt = load_text_file("prompts/jp-teacher.txt")
  model_id = 'amazon.titan-text-express-v1'

  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nBot:"
  resp = client.invoke_model(
      modelId=model_id,
      contentType='application/json',
      accept='application/json',
      body=json.dumps({
        "inputText": prompt
      })
  )
  body = json.loads(resp.get("body").read())
  llm_response = body['results'][0]['outputText']

  updated_history = f"{history}\n{llm_response}"
  Chat.write(session_id,updated_history)

  return body

In [30]:
print(converse("food","Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

Chat created.
{'inputTextTokenCount': 142, 'results': [{'tokenCount': 33, 'outputText': ' That sounds like a great idea! Kyoto Katsugyu is a popular Japanese restaurant in Toronto that serves delicious grilled meats. Are you excited to try it?', 'completionReason': 'FINISH'}]}


In [31]:
print(converse("food","How do I conjugate taberu?"))

Chat updated.
{'inputTextTokenCount': 165, 'results': [{'tokenCount': 128, 'outputText': ' Taberu is a Japanese verb that means "to eat." It is conjugated in the following way:\n\n- taberu - to eat\n- tabete - to eat\n- tabeta - to have eaten\n- tabeteimasu - to be eating\n- tabeteimasu - to have been eating\n- tabeteiru - to be eating\n- tabeteitai - to want to eat\n- tabeteitai - to have wanted to eat\n- tabeteimasen - to not eat\n- tabeteimasen - to have not eaten', 'completionReason': 'LENGTH'}]}


In [32]:
class JpAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [
    
    ] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    text = resp['output']['message']['content'][0]['text']      
    self.messages.append({"role": "assistant", "content": [{"text": text}]})
    return text
agent = JpAgent()

In [33]:
print(agent.chat("Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

Sure, let's translate that sentence into Japanese. Here are the key words with furigana readings and some clues:

行(い)く - "to go" (verb)
食(た)べる - "to eat" (verb)
食(く)い物(もの) - "food"
レストラン - "restaurant" (borrowed word)
京都(きょうと)かつ牛(ぎゅう) - "Kyoto Katsugyu" (name of the restaurant)
トロント - "Toronto" (place name)

Clue 1: The sentence starts with a suggestion using a volitional form verb.
Clue 2: You'll need a particle to connect "to go" and "to eat food".
Clue 3: Don't forget the particle indicating location after the restaurant name.

Give it a try! Let me know if you need any other clues.


In [34]:
print(agent.chat("How do I conjugate taberu?"))

Great question! For the verb 食べる (taberu) meaning "to eat", here are the different conjugations in the dictionary form:

食べる (taberu) - Dictionary form

Some common conjugations:
食べます (tabemasu) - Polite present tense
食べました (tabemashita) - Polite past tense  
食べません (tabemasen) - Polite negative 
食べませんでした (tabemasen deshita) - Polite negative past
食べて (tabete) - Te-form (connect to other verbs)
食べる (taberu) - Plain present 
食べた (tabeta) - Plain past
食べよう (tabeyou) - Volitional (let's...)

Let me know if you need any clarification on how these different forms are used in sentences.


In [37]:
class JpAgentStream:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse_stream(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    text = self.stream(resp)
    self.messages.append({"role": "assistant", "content": [{"text": text}]})
    return text
  def stream(self,resp):  
    full_response = ""
    for event in resp.get('stream'):
        if 'messageStart' in event:
            print("Response started:", event['messageStart']['role'])
        elif 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']['delta'].get('text', '')
            full_response += chunk
            print(chunk, end='', flush=True)  # Print chunks as they arrive
        elif 'messageStop' in event:
            print("\n\nResponse completed. Stop reason:", event['messageStop']['stopReason'])
        elif 'metadata' in event:
            print("\nMetadata:", json.dumps(event['metadata'], indent=2))  
    return full_response          
stream_agent = JpAgentStream()

In [38]:
stream_agent.chat("How do I conjugate taberu?")

Response started: assistant
Here are some clues to help you conjugate the Japanese verb 食べる (たべる, to eat):

The verb stem is 食べ (たべ).

To make the present affirmative form, you add -る to the stem.
食べる (たべる)

For the past affirmative, add -た to the stem.
食べた (たべた)

The negative is formed by adding -ない to the stem.
食べない (たべない)

The te-form (used for linking verbs) drops -ru and adds -て.
食べて (たべて)

Let me know if you need any other forms or have additional examples you'd like clues for conjugating.

Response completed. Stop reason: end_turn

Metadata: {
  "usage": {
    "inputTokens": 138,
    "outputTokens": 168,
    "totalTokens": 306
  },
  "metrics": {
    "latencyMs": 3960
  }
}


"Here are some clues to help you conjugate the Japanese verb 食べる (たべる, to eat):\n\nThe verb stem is 食べ (たべ).\n\nTo make the present affirmative form, you add -る to the stem.\n食べる (たべる)\n\nFor the past affirmative, add -た to the stem.\n食べた (たべた)\n\nThe negative is formed by adding -ない to the stem.\n食べない (たべない)\n\nThe te-form (used for linking verbs) drops -ru and adds -て.\n食べて (たべて)\n\nLet me know if you need any other forms or have additional examples you'd like clues for conjugating."